In [96]:
%matplotlib inline

import matplotlib
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import os

In [97]:
### FORMAT: [task, class, correct, task_nice, class_nice, correct_nice, id1, id1_nice, ..., idN, idN_nice]
tex_table_format =  \
'''\\begin{{figure}}[H]
  \centering
  \\begin{{subfigure}}[b]{{0.4\linewidth}}
    \includegraphics[width=\linewidth]{{images/{0}/{1}/{2}/{6}.png}}
    \caption{{Light Curve of \\textit{{{4}}} with {7}.}}
  \end{{subfigure}}
  \\begin{{subfigure}}[b]{{0.4\linewidth}}
    \includegraphics[width=\linewidth]{{images/{0}/{1}/{2}/{8}.png}}
    \caption{{Light Curve of \\textit{{{4}}} with {9}.}}
  \end{{subfigure}}
  \\begin{{subfigure}}[b]{{0.4\linewidth}}
    \includegraphics[width=\linewidth]{{images/{0}/{1}/{2}/{10}.png}}
    \caption{{Light Curve of \\textit{{{4}}} with {11}.}}
  \end{{subfigure}}
  \\begin{{subfigure}}[b]{{0.4\linewidth}}
    \includegraphics[width=\linewidth]{{images/{0}/{1}/{2}/{12}.png}}
    \caption{{Light Curve of \\textit{{{4}}} with {13}.}}
  \end{{subfigure}}
  \caption{{Light Curves of four {5} classified {4} sources in the {3} Classification task.}}
  \label{{fig:{0}_{1}_{2}}}
\end{{figure}}'''

In [98]:
DATA_PATH = '../../data/'
LIGHTCURVES_PATH = DATA_PATH + 'lightcurves/'

In [99]:
filename = 'nontransient_lightcurves.pickle'
indir = LIGHTCURVES_PATH; filepath = indir + filename
df_nontr = pd.read_pickle(filepath)
print(df_nontr.ID.unique().shape)

(16940,)


In [100]:
filename = 'transient_lightcurves.pickle'
indir = LIGHTCURVES_PATH; filepath = indir + filename
df_trans = pd.read_pickle(filepath)
print(df_trans.TransientID.unique().shape)

(4984,)


In [122]:
def plot_lightcurve(lc):
    mean = round(lc.Mag.mean())
    plot = lc.plot(x='MJD', y='Mag', marker='.', markeredgecolor='black', linestyle='None', legend=False, figsize=(7,7), ylim=(mean-4,mean+4), yerr='Magerr', elinewidth=0.7)
    plot.set_xlabel('Modified Julian Date')
    plot.set_ylabel('Mag')

In [123]:
def transient_lightcurve(objid):
    df_lc = df_trans[df_trans.TransientID == objid]
    return df_lc

def non_transient_lightcurve(objid):
    df_lc = df_nontr[df_nontr.ID == objid]
    return df_lc

In [124]:
def visualize_lcs(task, klass, correct, ids):
    for idx in ids:
        if 'Tran' in idx:
            lc = transient_lightcurve(idx)
        elif 'Cata' in idx:
            lc = non_transient_lightcurve(idx)
        plot_lightcurve(lc)
        outdir = '../../results/graphs/{}/{}/{}/'.format(task, klass, correct)
        if not os.path.exists(outdir):
            os.makedirs(outdir)
        plt.savefig(outdir + '{}.png'.format(idx))
        plt.close()

        
def tex_table(task, klass, correct, ids):
    
    task_nice = lambda x: x.replace('_', '-').title()
    class_nice = { 'transient': 'Transient', 'non-transient':'Non-Transient', 'SN':'Supernova' }
    correct_nice = lambda x: x + 'ly'
    id_nice = lambda x: 'Transient ID {}'.format(x[6:]) if 'Tran' in x else 'Catalina ID {}'.format(x[6:])
    
    ### Format: [task, class, correct, task_nice, class_nice, correct_nice, id1, id1_nice, ..., idN, idN_nice]
    # Tex Table params
    tex_table_params = [task, klass, correct, task_nice(task), class_nice.get(klass, klass), correct_nice(correct)]

    for idx in ids:
        tex_table_params.append(idx)
        tex_table_params.append(id_nice(idx))
        

    tex_table = tex_table_format.format(*tex_table_params)
    outdir = '../../results/graphs/{}/{}/{}/'.format(task, klass, correct)
    with open(outdir+'table.txt' ,'w') as f:
        f.write(tex_table)

In [138]:
for bool_correct in [True, False]:
    correct = 'correct' if bool_correct else 'incorrect'
    with open('../{}.txt'.format(correct) ,'r') as f:
        dict_from_file = eval(f.read())
    
    task = dict_from_file['task']
    dict_from_file.pop('task', None)
    
    for klass, ids in dict_from_file.items():
        visualize_lcs(task, klass, correct, ids)
        tex_table(task, klass, correct, ids)